In [4]:
import pyqstrat as pq
import h5py

def create_data():
    ohlcs = []

    with h5py.File('/Users/sal/Developer/ambrus/reports/tail_options/SPX.hdf5', 'r') as f:
        dates = np.arange(np.datetime64('2023-01-01'), np.datetime64('2023-01-10'))
        for date in dates:
            _date = pd.to_datetime(date).to_pydatetime()
            key = f'D_{_date:%Y_%m_%d}'
            if key not in f: continue
            _df = pq.hdf5_to_df('/Users/sal/Developer/ambrus/reports/tail_options/SPX.hdf5', key)
            df = _df.copy()
            df['expiry'] = pd.to_datetime(df.okey_yr * 10000 + df.okey_mn * 100 + df.okey_dy, format='%Y%m%d')
            df = df[['timestamp', 'okey_cp', 'expiry', 'okey_xx', 'prtPrice', 'prtSize', 'prtVolume', 'uBid', 'uAsk', 'prtIv', 'prtDe']]
            df['umid'] = 0.5 * (df.uBid + df.uAsk)
            del df['uBid']
            del df['uAsk']
            df.columns = ['timestamp', 'put_call', 'expiry', 'strike', 'prc', 'qty', 'v', 'iv', 'delta', 'umid']
            df['symbol'] = df.put_call.str[:1] + '-' + df.strike.astype(int).astype(str) + '-' + df.expiry.astype(str)
            df['timestamp'] = pd.to_datetime(df.timestamp)
            ohlc = df.set_index('timestamp').groupby(
                ['symbol'], as_index=False).resample('5min').agg(
                {'symbol': 'last', 
                 'prc': ['first', 'min', 'max', 'last'], 
                 'umid': 'last', 
                 'v': 'sum',
                 'iv': 'last',
                 'delta': 'last'})
            ohlc.reset_index(inplace=True)
            ohlc.columns = ['dummy', 'timestamp', 'symbol', 'o', 'h', 'l', 'c', 'umid', 'v', 'iv', 'delta']
            ohlc = ohlc[np.isfinite(ohlc.c)]
            ohlc = ohlc[['timestamp', 'symbol', 'o', 'h', 'l', 'c', 'v', 'umid', 'iv', 'delta']]
            #ohlc = ohlc[['timestamp', 'open', 'high', 'low', 'close']]
            ohlcs.append(ohlc)

    pd.concat(ohlcs).to_csv('support/spx_options.csv.gz', index=False)
    pd.read_csv('support/spx_options.csv.gz')


,timestamp,symbol,o,h,l,c,v,umid,iv,delta
0,2023-01-03 12:55:00,C-1000-2023-01-20,2800.29,2800.29,2800.82,2800.82,3,3801.025024,0.0000,0.0000
1,2023-01-03 12:45:00,C-1000-2023-06-16,2789.70,2788.15,2789.70,2788.67,150,3795.981934,0.0000,0.0000
2,2023-01-03 12:55:00,C-1000-2023-06-16,2788.87,2788.87,2788.87,2788.87,225,3796.055420,0.6625,0.9917
3,2023-01-03 12:05:00,C-1000-2023-12-15,2799.97,2799.97,2799.97,2799.97,1,3808.508057,0.5935,0.9811
4,2023-01-03 14:15:00,C-1000-2023-12-15,2798.02,2798.02,2798.02,2798.02,2,3807.986450,0.5457,0.9828
...,...,...,...,...,...,...,...,...,...,...
47926,2023-01-09 15:00:00,P-600-2025-12-19,3.80,3.80,3.80,3.80,1,3902.296021,0.4901,-0.0029
47927,2023-01-09 01:30:00,P-6000-2024-12-20,1736.90,1736.90,1736.90,1736.90,1,3906.978516,0.1757,-0.8904
47928,2023-01-09 09:30:00,P-800-2025-12-19,6.60,6.60,6.60,6.60,1,3913.342529,0.4508,-0.0053
47929,2023-01-09 15:30:00,P-900-2023-12-15,2.10,2.10,2.10,2.10,3,3892.508057,0.6205,-0.0025
